In [14]:
import pandas as pd

input_file='RawData/SemEval2014Task3/trial-data/'+'sentence2phrase.trial.input.txt'

inputs=pd.read_csv(input_file,sep='\t',header=None)
inputs.head()

,0,1
0,After the interval between the two halves of a...,association football half time
1,President Obama's push for military action to ...,average walk time for a 5k
2,The hard disk could not be used due to a perma...,bad sectors on hard disk
3,jumping from a tall structure while connected ...,bungee jumping
4,Race bike frames are made by extremely strong ...,carbon fiber bike frame


In [15]:
gs='RawData/SemEval2014Task3/evaluation/'+'sentence2phrase.trial.gs.txt'
gold=pd.read_csv(gs,sep='\t',header=None)
gold.head()

,0
0,1.75
1,0.00
2,3.25
3,4.00
4,3.75


In [18]:
input0=inputs[0].tolist()
input1=inputs[1].tolist()
simi=gold[0].tolist()

standards=pd.DataFrame({
    'Sent1':input0,
    'Sent2':input1,
    'Similarity':simi
})

standards.head()

,Sent1,Sent2,Similarity
0,After the interval between the two halves of a...,association football half time,1.75
1,President Obama's push for military action to ...,average walk time for a 5k,0.00
2,The hard disk could not be used due to a perma...,bad sectors on hard disk,3.25
3,jumping from a tall structure while connected ...,bungee jumping,4.00
4,Race bike frames are made by extremely strong ...,carbon fiber bike frame,3.75


In [25]:
cosineErr=[]
cosinePred=[]
error_sqr_sum=0
from gensim.parsing.preprocessing import remove_stopwords , preprocess_string

def cosine_distence(left,right):
    X_set=set(left)
    Y_set=set(right)
    l1=[]
    l2=[]
    rvector = X_set.union(Y_set)  
    for w in rvector:
        if w in X_set:l1.append(1)
        else:l1.append(0) 
        if w in Y_set:l2.append(1) 
        else:l2.append(0) 
    c=0
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        
    return cosine

for i in range(len(simi)):
    try:
        left=preprocess_string( remove_stopwords(input0[i]))
        right=preprocess_string( remove_stopwords(input1[i]))
        predicted=cosine_distence(left,right)*5
        error=simi[i]-predicted
        error_sqr_sum+=error*error
        cosineErr.append(error)
        cosinePred.append(predicted)
    except:
        continue

cosErrDf=pd.DataFrame(
{
    'Sent1':input0,
    'Sent2':input1,
    'Standard':simi,
    'Predicted':cosinePred,
    'Error':cosineErr
})

Cosmse=error_sqr_sum/len(input0) 

cosErrDf.head()


,Sent1,Sent2,Standard,Predicted,Error
0,After the interval between the two halves of a...,association football half time,1.75,0.000000,1.750000
1,President Obama's push for military action to ...,average walk time for a 5k,0.00,0.000000,0.000000
2,The hard disk could not be used due to a perma...,bad sectors on hard disk,3.25,2.236068,1.013932
3,jumping from a tall structure while connected ...,bungee jumping,4.00,1.336306,2.663694
4,Race bike frames are made by extremely strong ...,carbon fiber bike frame,3.75,3.015113,0.734887


In [26]:
print(Cosmse)

2.4012732901982186


In [57]:
#Finding MSE for our model  


import keras.backend as K
from keras.layers import Layer
from keras.optimizers import Adadelta
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
from gensim.parsing.preprocessing import remove_stopwords , preprocess_string
from gensim.models import Word2Vec
from numpy import array
import numpy as np

siMlErr=[]
siMlPred=[]
error_sqr_sum=0

class ManDist(Layer):
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)
    def call(self, x, **kwargs):
        self.result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
        return self.result
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)
kul=0
gul=0
tot=0
optimizer= Adadelta()
json_file = open("MetaData/"+"SiILstmModel3.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects ={'ManDist': ManDist})
# load weights into new model
loaded_model.load_weights('MetaData/'+'SiILstmModel4.h5')
mod3=loaded_model
mod3.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy','mean_squared_error','mean_absolute_percentage_error'])

#Load WordVec models
word_model = Word2Vec.load("MetaData/"+'Word2Vec.model')

for index, row in standards.iterrows():
    sentence1=row['Sent1']
    sentence2=row['Sent2']
    tokens1=preprocess_string( remove_stopwords(sentence1) )
    tokens2=preprocess_string( remove_stopwords(sentence2) )
    if tokens1==[]:tokens1=['b']
    if tokens2==[]:tokens2=['b']
    tot+=len(tokens1)
    tot+=len(tokens2)
    vec_matrix1=[]
    for word in tokens1:
        vec=0
        try:
            vec=word_model.wv[word]
            gul+=1
        except:
            vec=np.random.rand(300)
            kul+=1
        vec_matrix1.append(vec)
    vec_matrix2=[]
    for word in tokens2:
        vec=0
        try:
            vec=word_model.wv[word]
            gul+=1
        except:
            vec=np.random.rand(300)
            kul+=1
        vec_matrix2.append(vec)
    temp1=[vec_matrix1]
    temp2=[vec_matrix2]
  #  print('temp1\n\n',temp1[0][0],'\n\ntemp2\n',temp2[0][0])
    pad_vec_matrix1=pad_sequences(temp1, padding='post', truncating='post', maxlen=30 ,dtype='float64')
    pad_vec_matrix2=pad_sequences(temp2, padding='post', truncating='post', maxlen=30 ,dtype='float64')
    print(np.shape(pad_vec_matrix1),end='')
    prediction=mod3.predict([pad_vec_matrix1,pad_vec_matrix2])
    print(prediction,end='')
    prediction=prediction[0][0]*50
    error=row['Similarity']-prediction
    error_sqr_sum+=error*error
    siMlErr.append(error)
    siMlPred.append(prediction)


print(len(input1),len(siMlPred),len(siMlErr))        
siMlErrDf=pd.DataFrame(
{
    'Sent1':input0,
    'Sent2':input1,
    'Standard':simi,
    'Predicted':siMlPred,
    'Error':siMlErr
})
print('hajkdfksjf',gul,kul,tot)
siMlmse=error_sqr_sum/len(input0) 
print(siMlmse)
siMlErrDf[:30]

(1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932745]](1, 30, 300)[[-0.06915773]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[0.7060524]](1, 30, 300)[[-0.06787491]](1, 30, 300)[[-0.0686419]](1, 30, 300)[[-0.06931649]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06897139]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06921682]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06872776]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[-0.06932852]](1, 30, 300)[[0.15456131]](1, 30, 300)[[-0.06932341]](1, 30, 300)[[-0.06927677]](1, 30, 300)[[-0.01122617]](1, 30, 300)[[0.41475165]](1, 30, 300)[[-0.0693285]]36 36 36
hajkdfksjf 359 168 527
67

,Sent1,Sent2,Standard,Predicted,Error
0,After the interval between the two halves of a...,association football half time,1.750,-3.466426,5.216426
1,President Obama's push for military action to ...,average walk time for a 5k,0.000,-3.466426,3.466426
2,The hard disk could not be used due to a perma...,bad sectors on hard disk,3.250,-3.466426,6.716426
3,jumping from a tall structure while connected ...,bungee jumping,4.000,-3.466426,7.466426
4,Race bike frames are made by extremely strong ...,carbon fiber bike frame,3.750,-3.466373,7.216373
5,The first plans for a metro system in Moscow d...,cold war era,0.750,-3.457887,4.207887
6,"Historically, in countries where Communist Par...",Communist Party of the Soviet Union,2.000,-3.466426,5.466426
7,Vladimir Ilyich Lenin served as the leader of ...,Communist Party of the Soviet Union,2.000,-3.466426,5.466426
8,Hydroelectricity is the production of electric...,electricity generated by hydropower,4.000,-3.466426,7.466426
9,Researchers at the space agency believe they h...,Evidence for life in outer space,3.125,-3.466426,6.591426


[5.092215538024902,
 5.092096328735352,
 5.191850662231445,
 5.092072486877441,
 5.291604995727539,
 5.2916765213012695,
 5.191898345947266,
 5.191898345947266,
 5.291581153869629]